In [1]:
import utils
from torchvision import transforms
import my_dataset
import os
import torch
from tqdm import tqdm
from models.vit import ViT
from torch.utils.tensorboard import SummaryWriter

In [2]:
batch_size = 32
size = 32
lr = 2e-4
epochs = 150
save = True

In [3]:
writer = SummaryWriter(log_dir = 'logs')

In [4]:
train_data = utils.read_file("../cifar10/train_data.txt")
val_data = utils.read_file("../cifar10/val_data.txt")
test_data = utils.read_file("../cifar10/test_data.txt")
data_transform = {
        "train": transforms.Compose([
                                    transforms.RandomCrop(32, padding=4),
                                    transforms.Resize(size),
                                    transforms.RandomHorizontalFlip(),
                                     transforms.ToTensor(),
                                     transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])]),
        "val": transforms.Compose([transforms.ToTensor(),
                                   transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])}
train_dataset = my_dataset.MyDataSet_CIFAR(images_path=train_data,
                        transform=data_transform["train"])

val_dataset = my_dataset.MyDataSet_CIFAR(images_path=val_data,
                        transform=data_transform["val"])

In [5]:
nw = min([os.cpu_count(), batch_size if batch_size > 1 else 0, 8])  # number of workers
print('Using {} dataloader workers every process'.format(nw))

Using 8 dataloader workers every process


In [6]:
train_loader = torch.utils.data.DataLoader(train_dataset,
                                            batch_size = batch_size,
                                            shuffle=True,
                                            pin_memory=True,
                                            num_workers=nw,
                                            collate_fn=train_dataset.collate_fn)

val_loader = torch.utils.data.DataLoader(val_dataset,
                                            batch_size = batch_size,
                                            shuffle=True,
                                            pin_memory=True,
                                            num_workers=nw,
                                            collate_fn=val_dataset.collate_fn)

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [8]:
model = ViT(
    image_size = size,
    patch_size = 4,
    num_classes = 10,
    dim = 512,
    depth = 6,
    heads = 8,
    mlp_dim = 512,
    dropout = 0.1,
    emb_dropout = 0.1
).to(device)

In [9]:
input = torch.randn(1, 3, 32, 32).to(device)
writer.add_graph(model, input)

/home/lhw/anaconda3/envs/env1/lib/python3.9/site-packages/einops/einops.py:202: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  inferred_length: int = length // known_product


In [10]:
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)

In [11]:
tags = ["train_loss", "train_acc", "val_loss", "val_acc", "learning_rate"]
best_acc = 0

In [12]:
for epoch in range(epochs):
    
    model.train()
    accu_loss = torch.zeros(1).to(device)  # 累计损失
    accu_num = torch.zeros(1).to(device)  # 累计预测正确的样本数
    optimizer.zero_grad()

    sample_num = 0
    data_loader = tqdm(train_loader)
    for step, data in enumerate(data_loader):
        images, labels = data

        sample_num += images.shape[0]

        pred = model(images.to(device))
        
        pred_classes = torch.max(pred, dim=1)[1]  # 预测的类别，[1]是标签索引
       
        
        accu_num += torch.eq(pred_classes, labels.to(device)).sum()
        loss = loss_function(pred, labels.to(device))
        loss.backward()
        
        accu_loss += loss.detach()
        
        data_loader.desc = "[train epoch {}] loss: {:.3f}, acc: {:.3f}".format(epoch,
                                                                               accu_loss.item() / (step + 1),
                                                                               accu_num.item() / sample_num)
        optimizer.step()  # 更新
        optimizer.zero_grad()
    
    train_loss =  accu_loss.item() / (step + 1)
    train_acc = accu_num.item() / sample_num
    val_loss, val_acc = utils.evaluate(model=model,
                                data_loader=val_loader,
                                device=device,
                                epoch=epoch)
    writer.add_scalar(tags[0], train_loss, epoch)
    writer.add_scalar(tags[1], train_acc, epoch)
    writer.add_scalar(tags[2], val_loss, epoch)
    writer.add_scalar(tags[3], val_acc, epoch)
    writer.add_scalar(tags[4], optimizer.param_groups[0]["lr"], epoch)
    scheduler.step()
    if save:
        if (epoch+1)%10 == 0:
            torch.save(model.state_dict(), "./weights/model-{}.pth".format(epoch + 1))
        if best_acc <= val_acc:
            best_acc = val_acc
            print("The best model is save!")
            torch.save(model.state_dict(), "./weights/model-best.pth")

[valid epoch 0] loss: 1.600, acc: 0.415: 100%|██████████| 313/313 [00:05<00:00, 61.58it/s]


The best model is save!


[valid epoch 1] loss: 1.541, acc: 0.451: 100%|██████████| 313/313 [00:05<00:00, 61.50it/s]


The best model is save!


[valid epoch 2] loss: 1.416, acc: 0.491: 100%|██████████| 313/313 [00:05<00:00, 60.45it/s]


...

The best model is save!


[valid epoch 102] loss: 1.446, acc: 0.731: 100%|██████████| 313/313 [00:05<00:00, 60.87it/s]


The best model is save!


[train epoch 104] loss: 0.086, acc: 0.968:  57%|█████▋    | 619/1094 [00:30<00:23, 20.15it/s]


KeyboardInterrupt: 